In [25]:
import pandas as pd

df = pd.read_parquet("sih_rs_tratado.parquet")

In [29]:
import polars as pl
from pathlib import Path

# --- Configuração ---
# Ajuste o caminho se necessário
PROCESSED_FILE_PATH = Path("sih_rs_tratado.parquet")

def analisar_frequencia_etnia(caminho_arquivo: Path):
    """
    Lê um arquivo Parquet e conta a frequência dos 10 valores mais comuns
    na coluna 'ETNIA'.
    """
    if not caminho_arquivo.exists():
        print(f"ERRO: Arquivo não encontrado em '{caminho_arquivo}'")
        return

    print(f"Analisando o arquivo: {caminho_arquivo.name}")
    print("-" * 40)

    try:
        # 1. Lê o arquivo, selecionando apenas a coluna 'ETNIA' para economizar memória
        df = pl.read_parquet(caminho_arquivo, columns=["CID_NOTIF"])

        # 2. Conta a frequência de cada valor na coluna, ordena do mais frequente
        #    para o menos, e pega os 10 primeiros.
        contagem_etnia = (
            df.group_by("CID_NOTIF")
            .agg(pl.count().alias("contagem")) # Conta as ocorrências e renomeia para 'contagem'
            .sort("contagem", descending=True) # Ordena pela contagem
            .limit(10)                         # Pega os 10 primeiros
        )

        print("Contagem dos 10 valores mais frequentes para a coluna 'ETNIA':")
        print(contagem_etnia)

    except pl.ColumnNotFoundError:
        print("ERRO: A coluna 'ETNIA' não foi encontrada no arquivo.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

# --- Execução ---
if __name__ == "__main__":
    analisar_frequencia_etnia(PROCESSED_FILE_PATH)



Analisando o arquivo: sih_rs_tratado.parquet
----------------------------------------
Contagem dos 10 valores mais frequentes para a coluna 'ETNIA':
shape: (10, 2)
┌───────────┬──────────┐
│ CID_NOTIF ┆ contagem │
│ ---       ┆ ---      │
│ str       ┆ u32      │
╞═══════════╪══════════╡
│ 0         ┆ 11746494 │
│ Z302      ┆ 75210    │
│ O149      ┆ 40       │
│ O000      ┆ 21       │
│ O100      ┆ 21       │
│ O231      ┆ 19       │
│ O109      ┆ 18       │
│ O141      ┆ 11       │
│ O903      ┆ 11       │
│ O140      ┆ 7        │
└───────────┴──────────┘


/tmp/ipykernel_17885/900817837.py:28: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("contagem")) # Conta as ocorrências e renomeia para 'contagem'


In [26]:
df['ETNIA'].isnull().sum()

np.int64(0)

In [21]:
import pandas as pd
import polars as pl
from pathlib import Path

# Suponha que o seu DataFrame Polars já foi criado e está salvo
# no arquivo tratado (sih_rs_tratado.parquet)
caminho_arquivo = "sih_rs_tratado.parquet" 

try:
    # Lê o arquivo Parquet para um DataFrame Polars
    df_pl = pl.read_parquet(caminho_arquivo)
    
    # Converte o DataFrame do Polars para Pandas
    df_pd = df_pl.to_pandas()
    
    # Seleciona a coluna e conta os valores únicos
    coluna_interesse = 'NACIONAL'
    contagem_valores = df_pd[coluna_interesse].value_counts()
    
    # Define o nome do arquivo de saída
    nome_csv = f'contagem_{coluna_interesse}.csv'
    
    # Salva o resultado em um arquivo CSV
    contagem_valores.to_csv(nome_csv, header=True, index=True)
    
    print(f"Valores únicos da coluna '{coluna_interesse}' salvos em '{nome_csv}' com sucesso.")

except FileNotFoundError:
    print(f"Erro: O arquivo '{caminho_arquivo}' não foi encontrado. Certifique-se de que a etapa de pré-processamento foi executada com sucesso.")
except KeyError:
    print(f"Erro: A coluna '{coluna_interesse}' não existe no DataFrame.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Valores únicos da coluna 'NACIONAL' salvos em 'contagem_NACIONAL.csv' com sucesso.
